<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/LSTM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
with open('/content/hdfs_event2semantic_vec.json') as f:
    # Step1-1 open file
    gdp_list = json.load(f)
    value = list(gdp_list.values())

In [5]:
print(len(value))
print(len(value[0]))

29
300


In [19]:
components = None
pca = PCA(n_components = 20) # n_components can not be greater than 29 , because we have 29 data and if they are compeletely independent , we could have 29D, same as one hot encoding
# perform PCA on the scaled data
pca_result = pca.fit_transform(value)

# print the explained variances
print("Variances (Percentage):")
print(pca.explained_variance_ratio_ * 100)
print(sum(pca.explained_variance_ratio_ * 100)) #the percentage of variance for 20D


Variances (Percentage):
[26.38436577 11.54957801  8.87619262  8.09854493  5.83096291  4.95880394
  4.05582561  3.81095343  3.24716904  2.76424442  2.52277546  2.38701665
  1.98277494  1.86786663  1.6366585   1.51299209  1.47744943  1.28605957
  1.2501469   0.99433261]
96.49471344390082


In [20]:
print(pca_result.shape)

(29, 20)


In [26]:
with open('/content/hdfs_event2semantic_vec.json') as f:
    # Step1-1 open file
    gdp_list = json.load(f)
    value = list(gdp_list.values())

    # Step1-2 PCA: Dimensionality reduction to 20-dimensional data
    from sklearn.decomposition import PCA
    estimator = PCA(n_components=20)
    pca_result = estimator.fit_transform(value)

    # Step1-3 PPA: De-averaged
    ppa_result = []
    result = pca_result - np.mean(pca_result)
    pca = PCA(n_components=20)
    pca_result = pca.fit_transform(result)
    U = pca.components_
    for i, x in enumerate(result):
        for u in U[0:7]:
            x = x - np.dot(u.transpose(), x) * u
        ppa_result.append(list(x))
    ppa_result = np.array(ppa_result)


In [ ]:
logs_series = pd.read_csv(path)
logs_series = logs_series.values
label = logs_series[:,1]
logs_data = logs_series[:,0]
logs = []
for i in range(0,len(logs_data)):
    padding = np.zeros((300,20))
    data = logs_data[i]
    data = [int(n) for n in data.split()]
    for j in range(0,len(data)):
        padding[j] = ppa_result[data[j]]
    padding = list(padding)
    logs.append(padding)
logs = np.array(logs)
split_boundary = int(logs.shape[0] * split)
train_x = logs[: split_boundary]
valid_x = logs[split_boundary:]
train_y = label[: split_boundary]
valid_y = label[split_boundary:]
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 20))
valid_x = np.reshape(valid_x, (valid_x.shape[0], valid_x.shape[1], 20))
train_y = keras.utils.to_categorical(np.array(train_y))
valid_y = keras.utils.to_categorical(np.array(valid_y))